# Finetune ruGPT3Small on your data


## Install enviroment

In [ ]:
!pip3 install urllib3==1.25.4

In [ ]:


# opening the file
file_obj = open("jokes.txt", "r")

# reading the data from the file
file_data = file_obj.read()

# splitting the file data into lines
lines = file_data.splitlines()
file_obj.close()
len(lines)

In [ ]:
with open("train_censored.txt", "w") as file:
    file.write("\n".join(lines[:100000]))

In [ ]:
with open("valid_censored.txt", "w") as file:
    file.write("\n".join(lines[100000:]))

In [ ]:
!pip3 install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!unzip /content/rugpt3_small_10epochs_50ktrain_args.zip -d /content/rugpt3_small_10epochs_50ktrain/

## Add data to colab


In [ ]:
data_path = "/content/drive/MyDrive/diploma/russian_jokes.csv"

## Prepare data

In [ ]:
import pandas as pd
data = pd.read_csv(data_path)
data.head()

In [ ]:
import numpy as np
import random

In [ ]:
random.seed(22)
np.random.seed(22)

In [ ]:

data.shape

In [ ]:
val_ind = random.sample(range(data.shape[0]), 25000)

In [ ]:
train = [data.iloc[i]['0'] for i in range(len(data)) if i not in val_ind][:75000]
valid = [data.iloc[i]['0'] for i in range(len(data)) if i in val_ind]

In [ ]:
len(train), len(valid)

In [ ]:
with open("train1.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid1.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = tensor(13.8065)

In [ ]:
!python pretrain.py \
    --output_dir=jokes_model \
    --model_type=gpt2 \
    --model_name_or_path=ai-forever/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train_censored.txt \
    --do_eval \
    --eval_data_file=valid_censored.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 15 \
    --block_size 2048 \
    --overwrite_output_dir

In [ ]:
val_ind = random.sample(range(data.shape[0]), 20000)
train = [data.iloc[i]['0'] for i in range(len(data)) if i not in val_ind][:230000]
valid = [data.iloc[i]['0'] for i in range(len(data)) if i in val_ind]
with open("valid1.txt", "w") as file:
    file.write("\n".join(valid))
with open("train1.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
!python pretrain.py \
    --output_dir=jokes_model \
    --model_type=gpt2 \
    --model_name_or_path=ai-forever/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train1.txt \
    --do_eval \
    --eval_data_file=valid1.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 15 \
    --block_size 2048 \
    --overwrite_output_dir

## Check our model

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=comment_model \
    --temperature=1.5 \
    --k=8 \
    --p=0.95 \
    --prompt="Приходит Илья Кречин к Никите и говорит: " \
    --length=28

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=model \
    --temperature=3 \
    --k=10 \
    --p=0.95 \
    --prompt="Приходит Штирлиц" \
    --length=28

In [ ]:
!cd /content/comment_model
!zip -r rugpt3_small_3epochs_200ktrain.zip comment_model/checkpoint-5500/

In [ ]:
!cd /content/jokes_model
!zip -r /content/drive/MyDrive/diploma/rugpt3_small_15epochs_100ktrain.zip jokes_model/checkpoint-20000


In [ ]:
from google.colab import files
files.download('dataset.zip')